In [318]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import shutil
from sklearn.model_selection import train_test_split
import random
import matplotlib.image as mpimg
from tensorflow.keras.optimizers import Adam # type: ignore
from tensorflow.keras.preprocessing.image import ImageDataGenerator # type: ignore
from tensorflow.keras.optimizers import SGD # type: ignore
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping # type: ignore
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model  
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.preprocessing.image import load_img, img_to_array #type: ignore

In [274]:
data_dir = "/Users/michelangelozampieri/Desktop/TAMID-group/data/seven_plastics"

In [275]:
classes = []
for root, dirs, files in os.walk(data_dir):
    for name in dirs:
        classes.append(name)
print(classes)

['4_low_density_polyethylene_PE-LD', '8_no_plastic', '3_polyvinylchloride_PVC', '1_polyethylene_PET', '6_polystyrene_PS', '5_polypropylene_PP', '7_other_resins', '2_high_density_polyethylene_PE-HD']


## Split directories into train, test, and validation

In [276]:
output_dir = "/Users/michelangelozampieri/Desktop/TAMID-group/data/seven_plastics_output"

train_dir = os.path.join(output_dir, "train")
validation_dir = os.path.join(output_dir, "validation")
test_dir = os.path.join(output_dir, "test")

os.makedirs(train_dir, exist_ok=True)
os.makedirs(validation_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

In [277]:
for class_name in classes:
    class_dir = os.path.join(data_dir, class_name)
    images = os.listdir(class_dir)

    train_val_images, test_images = train_test_split(images, test_size=0.15, random_state=42)
    train_images, validation_images = train_test_split(train_val_images, test_size=0.1765, random_state=42)

    os.makedirs(os.path.join(train_dir, class_name), exist_ok=True)
    os.makedirs(os.path.join(validation_dir, class_name), exist_ok=True)
    os.makedirs(os.path.join(test_dir, class_name), exist_ok=True)

    for image in train_images:
        shutil.copy(os.path.join(class_dir, image), os.path.join(train_dir, class_name, image))
    for image in validation_images:
        shutil.copy(os.path.join(class_dir, image), os.path.join(validation_dir, class_name, image))
    for image in test_images:
        shutil.copy(os.path.join(class_dir, image), os.path.join(test_dir, class_name, image))

In [278]:
print("Number of images in training directory:")
for class_name in classes:
    print(f"{class_name}: {len(os.listdir(os.path.join(train_dir, class_name)))}")
print("Number of images in validation directory:")
for class_name in classes:
    print(f"{class_name}: {len(os.listdir(os.path.join(test_dir, class_name)))}")
print("Number of images in test directory:")
for class_name in classes:
    print(f"{class_name}: {len(os.listdir(os.path.join(validation_dir, class_name)))}")

Number of images in training directory:
4_low_density_polyethylene_PE-LD: 4
8_no_plastic: 3
3_polyvinylchloride_PVC: 1
1_polyethylene_PET: 4
6_polystyrene_PS: 1
5_polypropylene_PP: 8
7_other_resins: 3
2_high_density_polyethylene_PE-HD: 3
Number of images in validation directory:
4_low_density_polyethylene_PE-LD: 2
8_no_plastic: 1
3_polyvinylchloride_PVC: 1
1_polyethylene_PET: 2
6_polystyrene_PS: 1
5_polypropylene_PP: 2
7_other_resins: 1
2_high_density_polyethylene_PE-HD: 1
Number of images in test directory:
4_low_density_polyethylene_PE-LD: 1
8_no_plastic: 1
3_polyvinylchloride_PVC: 1
1_polyethylene_PET: 2
6_polystyrene_PS: 1
5_polypropylene_PP: 2
7_other_resins: 1
2_high_density_polyethylene_PE-HD: 1


## Visualize some images

In [279]:
def plot_images(class_name):
    class_dir = os.path.join(train_dir, class_name)
    images = os.listdir(class_dir)
    random_images = random.sample(images, 1)
    plt.figure(figsize=(15, 5))
    for i, image in enumerate(random_images):
        plt.subplot(1, 3, i+1)
        img = mpimg.imread(os.path.join(class_dir, image))
        plt.imshow(img)
        plt.axis("off")
    plt.show()

# for c in classes:
#     print (f"Images for class {c} in training directory:")
#     plot_images(c)

## Define data augmentation

Lots of data augmentation to make up for small data set

In [280]:
def add_gaussian_noise(image):
    noise = np.random.normal(loc=0.0, scale=0.1, size=image.shape)  # Adjust scale as needed
    image = image + noise
    image = np.clip(image, 0.0, 1.0)  # Ensure pixel values remain valid
    return image

def custom_augmentation(image):
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_brightness(image, 0.2)
    image = tf.image.random_contrast(image, 0.8, 1.2)
    image = tf.image.random_hue(image, 0.2)
    image = tf.image.random_saturation(image, 0.8, 1.2)
    image = tf.image.random_crop(image, size=[100, 100, 3])  # Randomly crop the image
    image = tf.image.resize(image, [224, 224])  # Resize back to the target size
    image = tf.image.random_jpeg_quality(image, 75, 100)  # Randomly adjust JPEG quality
    image = tf.image.adjust_gamma(image, gamma=0.8)  # Adjust gamma
    image = tf.image.random_saturation(image, 0.6, 1.4)  # Further randomize saturation
    image = tf.image.random_hue(image, 0.3)  # Further randomize hue
    image = tf.image.random_brightness(image, 0.3)  # Further randomize brightness
    image = tf.image.random_contrast(image, 0.7, 1.3)  # Further randomize contrast
    image = add_gaussian_noise(image)  # Add Gaussian noise
    return image

In [281]:
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    preprocessing_function=custom_augmentation
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical"
)

Found 27 images belonging to 8 classes.


In [282]:
validation_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical"
)

Found 10 images belonging to 8 classes.


In [283]:
test_dataset = tf.keras.utils.image_dataset_from_directory(
    "/Users/michelangelozampieri/Desktop/TAMID-group/data/seven_plastics_output/test",
    labels="inferred",  
    label_mode="categorical",  
    image_size=(224, 224)
)

Found 11 files belonging to 8 classes.


## Define, compile and train base model

First build a model from scratch

In [284]:
model = tf.keras.models.Sequential([
    # First convolutional layer with 32 filters, 3x3 kernel size, ReLU activation, and input shape of 224x224 pixels with 3 color channels
    tf.keras.layers.Conv2D(32, (3, 3), activation="relu", input_shape=(224, 224, 3)),
    # First max pooling layer with a 2x2 pool size
    tf.keras.layers.MaxPooling2D(2, 2),
    # Second convolutional layer with 64 filters and 3x3 kernel size
    tf.keras.layers.Conv2D(64, (3, 3), activation="relu"),
    # Second max pooling layer with a 2x2 pool size
    tf.keras.layers.MaxPooling2D(2, 2),
    # Third convolutional layer with 128 filters and 3x3 kernel size
    tf.keras.layers.Conv2D(128, (3, 3), activation="relu"),
    # Third max pooling layer with a 2x2 pool size
    tf.keras.layers.MaxPooling2D(2, 2),
    # Fourth convolutional layer with 128 filters and 3x3 kernel size
    tf.keras.layers.Conv2D(128, (3, 3), activation="relu"),
    # Fourth max pooling layer with a 2x2 pool size
    tf.keras.layers.MaxPooling2D(2, 2),
    # Flatten layer to convert 3D feature maps to 1D feature vectors
    tf.keras.layers.Flatten(),
    # Fully connected dense layer with 512 units and ReLU activation
    tf.keras.layers.Dense(512, activation="relu", kernel_regularizer=tf.keras.regularizers.l2(0.01)),
    # Output dense layer with 8 units (one for each class) and softmax activation
    tf.keras.layers.Dense(8, activation="softmax")
])

/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [285]:
model.compile(
    loss="categorical_crossentropy",
    optimizer='rmsprop',
    metrics=["accuracy"]
)

lr_schedule = tf.keras.callbacks.LearningRateScheduler(lambda epoch: 1e-4 * 10**(-epoch / 10))

In [286]:
history = model.fit(
    train_generator,
    epochs=20,
    validation_data=validation_generator,
    callbacks=[lr_schedule]
)

/opt/anaconda3/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.1111 - loss: 12.0404 - val_accuracy: 0.1000 - val_loss: 11.9208 - learning_rate: 1.0000e-04
Epoch 2/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.2963 - loss: 11.9177 - val_accuracy: 0.1000 - val_loss: 11.8264 - learning_rate: 7.9433e-05
Epoch 3/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.2963 - loss: 11.8243 - val_accuracy: 0.1000 - val_loss: 11.7532 - learning_rate: 6.3096e-05
Epoch 4/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.2963 - loss: 11.7527 - val_accuracy: 0.2000 - val_loss: 11.6965 - learning_rate: 5.0119e-05
Epoch 5/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 829ms/step - accuracy: 0.2963 - loss: 11.6973 - val_accuracy: 0.2000 - val_loss: 11.6523 - learning_rate: 3.9811e-05
Epoch 6/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 757ms/step - accuracy: 0.2963 - loss: 11.6543 - val_accuracy: 0.2000 - val_loss: 11.6179 - learning_rate: 3.1623e-05
Epoch 7/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 790ms/step - accuracy: 0.2963 - loss:

In [287]:
val_loss, val_accuracy = model.evaluate(test_dataset)
print(f"Validation accuracy: {val_accuracy}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 234ms/step - accuracy: 0.1818 - loss: 50.3911
Validation accuracy: 0.1818181872367859


### `from_scratch_model.h5` has test accuracy of 9%

In [288]:
model.save("from_scratch_model.h5")

In [289]:
model = tf.keras.models.load_model("from_scratch_model.h5")

## Fine Tune the model

In [290]:
for layer in model.layers[:-10]:  # Freeze all layers except the last 10
    layer.trainable = False 

model.compile(optimizer=SGD(learning_rate=1e-4, momentum=0.9),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

lr_schedule = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, verbose=1, min_lr=1e-7)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [291]:
lepochs = 20  

history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=20,
    callbacks=[lr_schedule, early_stopping]
)

Epoch 1/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.2963 - loss: 11.5024 - val_accuracy: 0.2000 - val_loss: 11.4953 - learning_rate: 1.0000e-04
Epoch 2/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.2963 - loss: 11.5024 - val_accuracy: 0.2000 - val_loss: 11.4952 - learning_rate: 1.0000e-04
Epoch 3/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 816ms/step - accuracy: 0.2963 - loss: 11.5023 - val_accuracy: 0.2000 - val_loss: 11.4951 - learning_rate: 1.0000e-04
Epoch 4/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 847ms/step - accuracy: 0.2963 - loss: 11.5021 - val_accuracy: 0.2000 - val_loss: 11.4950 - learning_rate: 1.0000e-04
Epoch 5/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 797ms/step - accuracy: 0.2963 - loss: 11.5019 - val_accuracy: 0.2000 - val_loss: 11.4948 - learning_rate: 1.0000e-04
Epoch 6/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 745ms/step - accuracy: 0.2963 - loss: 11.5017 - val_accuracy: 0.2000 - val_loss: 11.4946 - learning_rate: 1.0000e-04
Epoch 7/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 687ms/step - accuracy: 0.2963 -

In [292]:
val_loss, val_accuracy = model.evaluate(test_dataset)
print(f"Validation accuracy: {val_accuracy}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - accuracy: 0.1818 - loss: 51.0054
Validation accuracy: 0.1818181872367859


### `fine_tuned_from_scratch_model.h5` has test accuracy of 9%

In [293]:
model.save("fine_tuned_from_scratch.h5")

## Using Efficienetb0

In [294]:
from tensorflow.keras.applications import EfficientNetB0 # type: ignore
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, BatchNormalization # type: ignore
from tensorflow.keras.models import Model # type: ignore

In [295]:
base_model = EfficientNetB0(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Initially freeze the base model

# Custom classification head
x = GlobalAveragePooling2D()(base_model.output)  # Reduce feature maps to a single vector
x = Dense(64, activation="relu")(x)  # Fewer neurons for simplicity
x = BatchNormalization()(x)  # Optional: Stabilize training
output_layer = Dense(8, activation="softmax")(x)  # Output layer

# Create model
model = Model(inputs=base_model.input, outputs=output_layer)

In [296]:
model.compile(optimizer="adam", 
              loss="categorical_crossentropy", 
              metrics=["accuracy"])

history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=50
)

Epoch 1/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step - accuracy: 0.1481 - loss: 2.2560 - val_accuracy: 0.1000 - val_loss: 2.0949
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 695ms/step - accuracy: 0.1481 - loss: 2.0124 - val_accuracy: 0.1000 - val_loss: 2.1042
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 770ms/step - accuracy: 0.1481 - loss: 2.0923 - val_accuracy: 0.1000 - val_loss: 2.1028
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 925ms/step - accuracy: 0.0741 - loss: 2.1542 - val_accuracy: 0.1000 - val_loss: 2.1035
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 797ms/step - accuracy: 0.1111 - loss: 2.2557 - val_accuracy: 0.1000 - val_loss: 2.1048
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 754ms/step - accuracy: 0.0741 - loss: 2.1240 - val_accuracy: 0.1000 - val_loss: 2.1050
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 683ms/step - accuracy: 0.0741 - loss: 2.2313 - val_accuracy: 0.1000 - val_loss: 2.1044
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 666ms/step - accuracy: 0.1481 - loss: 2.2117 - val_accuracy: 0.1000 - val_loss: 2.

In [297]:
val_loss, val_accuracy = model.evaluate(test_dataset)
print(f"Validation accuracy: {val_accuracy}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step - accuracy: 0.1818 - loss: 2.2014
Validation accuracy: 0.1818181872367859


### `efficientnetb0_base.h5` has test accuracy of 18%

In [298]:
model.save("efficientnetb0_base.h5")

In [299]:
model.trainable = True
for layer in model.layers[:-20]:  # Freeze all layers except the last 20
    layer.trainable = False

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)

model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])

history_fine = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=50
)

Epoch 1/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step - accuracy: 0.2222 - loss: 2.0642 - val_accuracy: 0.2000 - val_loss: 2.1925
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 736ms/step - accuracy: 0.2593 - loss: 2.1382 - val_accuracy: 0.2000 - val_loss: 2.1902
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 669ms/step - accuracy: 0.2963 - loss: 1.9329 - val_accuracy: 0.2000 - val_loss: 2.1880
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 894ms/step - accuracy: 0.1852 - loss: 2.1510 - val_accuracy: 0.2000 - val_loss: 2.1859
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 809ms/step - accuracy: 0.2222 - loss: 2.0819 - val_accuracy: 0.2000 - val_loss: 2.1839
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 748ms/step - accuracy: 0.2222 - loss: 2.0439 - val_accuracy: 0.2000 - val_loss: 2.1821
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 691ms/step - accuracy: 0.2963 - loss: 1.9808 - val_accuracy: 0.2000 - val_loss: 2.1804
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 743ms/step - accuracy: 0.2593 - loss: 2.0572 - val_accuracy: 0.2000 - val_loss: 2.

In [300]:
val_loss, val_accuracy = model.evaluate(test_dataset)
print(f"Validation accuracy: {val_accuracy}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 138ms/step - accuracy: 0.1818 - loss: 2.4872
Validation accuracy: 0.1818181872367859


### `fine_tuned_efficientNetb0.h5` has test accuracy of 18%

In [301]:
model.save("fine_tuned_efficientnetb0.h5")

## Using ResNet50

In [302]:
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = GlobalAveragePooling2D()(x)  # Pooling layer
x = Dense(256, activation='relu')(x)  # Fully connected layer
x = Dropout(0.5)(x)  # Dropout for regularization
predictions = Dense(8, activation='softmax')(x)  # Output layer

classifier = Model(inputs=base_model.input, outputs=predictions)

In [303]:
classifier.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [304]:
history = classifier.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=50,
    steps_per_epoch=len(train_generator),
    validation_steps=len(validation_generator)
)

Epoch 1/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step - accuracy: 0.1111 - loss: 2.1720 - val_accuracy: 0.2000 - val_loss: 2.1791
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.2963 - loss: 2.3401 - val_accuracy: 0.2000 - val_loss: 2.2856
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.1481 - loss: 2.3721 - val_accuracy: 0.2000 - val_loss: 2.3132
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.2593 - loss: 2.0820 - val_accuracy: 0.2000 - val_loss: 2.2909
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.0741 - loss: 2.2850 - val_accuracy: 0.2000 - val_loss: 2.2494
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.1481 - loss: 2.0000 - val_accuracy: 0.2000 - val_loss: 2.2188
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.2222 - loss: 2.1103 - val_accuracy: 0.2000 - val_loss: 2.1965
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.2963 - loss: 1.9608 - val_accuracy: 0.2000 - val_loss: 2.1837
Epoch 9/50
1/1 ━

In [305]:
val_loss, val_accuracy = classifier.evaluate(test_dataset)
print(f"Validation accuracy: {val_accuracy}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 351ms/step - accuracy: 0.0000e+00 - loss: 2.7327
Validation accuracy: 0.0


### `ResNet50_base` has test accuracy of 9%

In [306]:
classifier.save("resnet50_base.h5")

In [308]:
res = load_model("/Users/michelangelozampieri/Desktop/TAMID-group/models/resnet50_custom_classifier.h5")

res.summary()

Model: "sequential_17"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten_11 (Flatten)            │ (None, 100352)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_55 (Dense)                │ (None, 512)            │    51,380,736 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_21 (Dropout)            │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_56 (Dense)                │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_22 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_57 (Dense)                │ (None, 8)              │         2,056 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 51,514,122 (196.51 MB)

 Trainable params: 51,514,120 (196.51 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [309]:
# Unfreeze the last few layers of the base model
for layer in base_model.layers[-10:]:  # Unfreeze the last 10 layers
    layer.trainable = True

# Recompile the model with a lower learning rate for fine-tuning
classifier.compile(optimizer=Adam(learning_rate=1e-5),  # Lower learning rate
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [310]:
history_fine_tune = classifier.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=50,
    steps_per_epoch=len(train_generator),
    validation_steps=len(validation_generator)
)

Epoch 1/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step - accuracy: 0.2222 - loss: 2.0291 - val_accuracy: 0.2000 - val_loss: 2.1251
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.1852 - loss: 1.9974 - val_accuracy: 0.2000 - val_loss: 2.1249
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.2222 - loss: 1.9645 - val_accuracy: 0.2000 - val_loss: 2.1249
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.2963 - loss: 1.9422 - val_accuracy: 0.2000 - val_loss: 2.1247
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.2963 - loss: 1.8660 - val_accuracy: 0.2000 - val_loss: 2.1246
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.2593 - loss: 2.0093 - val_accuracy: 0.2000 - val_loss: 2.1244
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.2963 - loss: 1.9503 - val_accuracy: 0.2000 - val_loss: 2.1241
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.2593 - loss: 1.9692 - val_accuracy: 0.2000 - val_loss: 2.1237
Epoch 9/50
1/1 ━

In [311]:
val_loss, val_accuracy = classifier.evaluate(test_dataset)
print(f"Validation accuracy: {val_accuracy}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 369ms/step - accuracy: 0.0000e+00 - loss: 2.8337
Validation accuracy: 0.0


### `resNet50_fine_tuned.h5` has test accuracy of 18%

In [312]:
classifier.save("reseNet50_fine_tuned.h5")

In [313]:
res = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

for layer in base_model.layers:
    layer.trainable = False

In [319]:
def extract_features(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    features = base_model.predict(img_array)
    return features

# Example: Extract features for all images in a directory
def extract_features_for_dataset(dataset_dir):
    features = []
    labels = []
    for class_name in os.listdir(dataset_dir):
        class_dir = os.path.join(dataset_dir, class_name)
        if os.path.isdir(class_dir):
            for img_name in os.listdir(class_dir):
                img_path = os.path.join(class_dir, img_name)
                feature = extract_features(img_path)
                features.append(feature)
                labels.append(class_name)
    return np.array(features), np.array(labels)

In [320]:
# Extract features for training data
train_features, train_labels = extract_features_for_dataset(train_dir)

# Extract features for validation data
val_features, val_labels = extract_features_for_dataset(validation_dir)

# Reshape features (if necessary)
train_features = train_features.reshape(train_features.shape[0], -1)
val_features = val_features.reshape(val_features.shape[0], -1)

# Save the corresponding labels
train_labels = train_generator.classes
val_labels = validation_generator.classes

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 902ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 147ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━

In [321]:
train_labels = to_categorical(train_labels, num_classes=train_generator.num_classes)
val_labels = to_categorical(val_labels, num_classes=validation_generator.num_classes)

In [324]:
# Build a custom classifier
classifier = Sequential([
    Flatten(input_shape=train_features.shape[1:]),
    Dense(512, activation='relu', kernel_regularizer=l2(0.01)),  # L2 regularization
    Dropout(0.5),
    Dense(256, activation='relu', kernel_regularizer=l2(0.01)),
    Dropout(0.5),
    Dense(train_generator.num_classes, activation='softmax') # Output layer
])

# Compile the classifier
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [325]:
history = classifier.fit(
    train_features, train_labels,
    validation_data=(val_features, val_labels),
    epochs=50,
    batch_size=32,
)

Epoch 1/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.0000e+00 - loss: 19.9001 - val_accuracy: 0.2000 - val_loss: 63.6574
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 520ms/step - accuracy: 0.3333 - loss: 54.5986 - val_accuracy: 0.2000 - val_loss: 58.3595
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 308ms/step - accuracy: 0.5185 - loss: 31.1466 - val_accuracy: 0.2000 - val_loss: 54.7496
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 417ms/step - accuracy: 0.4815 - loss: 28.4881 - val_accuracy: 0.2000 - val_loss: 43.2125
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 268ms/step - accuracy: 0.5926 - loss: 30.5676 - val_accuracy: 0.2000 - val_loss: 63.3410
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 326ms/step - accuracy: 0.7778 - loss: 25.0529 - val_accuracy: 0.2000 - val_loss: 83.3094
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 544ms/step - accuracy: 0.6296 - loss: 27.4070 - val_accuracy: 0.3000 - val_loss: 81.3096
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 323ms/step - accuracy: 0.7778 - loss: 28.7512 - val_accuracy: 0.

In [326]:
val_loss, val_accuracy = classifier.evaluate(val_features, val_labels)
print(f"Validation Accuracy: {val_accuracy * 100:.2f}%")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step - accuracy: 0.4000 - loss: 109.7661
Validation Accuracy: 40.00%


### `ResNet50_base_v2.h5` has test accuracy of 20%

In [327]:
classifier.save("resnet50_custom_classifier.h5")

## Compare the models

In [329]:
model_dir = "/Users/michelangelozampieri/Desktop/TAMID-group/models"
model_files = os.listdir(model_dir)
model_files = [file for file in model_files if (file.endswith(".h5")| file.endswith(".keras"))]

model_data = []
for model_file in model_files:
    try:
        model = tf.keras.models.load_model(os.path.join(model_dir, model_file))
        val_loss, val_accuracy = model.evaluate(test_dataset)
        model_data.append({"Model": model_file, "Accuracy": val_accuracy})
    except Exception as e:
        print(f"Could not evaluate model {model_file}: {e}")

model_df = pd.DataFrame(model_data)
model_df = model_df.sort_values("Accuracy", ascending=False)
print(model_df)

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.0000e+00 - loss: 2.7327


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 247ms/step - accuracy: 0.1818 - loss: 50.3911


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 244ms/step - accuracy: 0.1818 - loss: 51.0054


Could not evaluate model resnet50_custom_classifier.h5: Exception encountered when calling Sequential.call().

Invalid input shape for input Tensor("data:0", shape=(None, 224, 224, 3), dtype=float32). Expected shape (None, 100352), but input has incompatible shape (None, 224, 224, 3)

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(None, 224, 224, 3), dtype=float32)
  • training=False
  • mask=None


1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.1818 - loss: 2.4872


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.0000e+00 - loss: 2.8337


1/1 ━━━━━━━━━━━━━━━━━━━━ 9s 9s/step - accuracy: 0.1818 - loss: 2.2014
                          Model  Accuracy
1         from_scratch_model.h5  0.181818
2    fine_tuned_from_scratch.h5  0.181818
3  fine_tuned_efficientnetb0.h5  0.181818
5        efficientnetb0_base.h5  0.181818
0              resnet50_base.h5  0.000000
4       reseNet50_fine_tuned.h5  0.000000


In [331]:
best_model = load_model("/Users/michelangelozampieri/Desktop/TAMID-group/models/resnet50_custom_classifier.h5")

best_model.summary()

Model: "sequential_20"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten_14 (Flatten)            │ (None, 100352)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_67 (Dense)                │ (None, 512)            │    51,380,736 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_26 (Dropout)            │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_68 (Dense)                │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_27 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_69 (Dense)                │ (None, 8)              │         2,056 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 51,514,122 (196.51 MB)

 Trainable params: 51,514,120 (196.51 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [333]:
def predict_image(model, img_path):
    img = load_img(img_path, target_size=(224, 224))
    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)
    prediction = model.predict(img_array)
    prediction = prediction[0]
    predicted_class = np.argmax(prediction)
    predicted_class_name = classes[predicted_class]
    plt.imshow(img)
    plt.axis("off")
    plt.title(f"Predicted: {predicted_class_name}\nActual: {os.path.basename(os.path.dirname(img_path))}")
    plt.show()

# Select random images from test directories
random_images = []
for class_name in os.listdir(test_dir):
    class_dir = os.path.join(test_dir, class_name)
    if os.path.isdir(class_dir):
        images = os.listdir(class_dir)
        if images:
            random_images.append(os.path.join(class_dir, random.choice(images)))

In [334]:
for img_path in random_images:
    predict_image(best_model, img_path)

ValueError: Exception encountered when calling Sequential.call().

[1mInvalid input shape for input Tensor("data:0", shape=(1, 224, 224, 3), dtype=float32). Expected shape (None, 100352), but input has incompatible shape (1, 224, 224, 3)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(1, 224, 224, 3), dtype=float32)
  • training=False
  • mask=None